# The Posterior Predictive Distribution by Sampling 

## Motivation
In this notebook we will explain how to obtain an approximate predictive distribution sampling. The two main reasons that we want to obtain the posterior predictive distribution are:

- to predict or to forecast new data 
- to check the model fit using what are known as "posterior predictive checks"

The last point is a very valuable and important task that we must consider doing whenever we fit a model in the bayesian paradigm.

For the posterior predictive distribution there is an exact equation, but importantly in most applied circumstances this equation requires intractable calculations to be done. This implies that we cannot obtain the exact posterior predictive distribution and thus we can overcome this difficulty by relying on a sampling approach.

To start with, a posterior predictive distribution is the probability distribution over some new data denoted as $\tilde{x}$ given that we have observed our sample $X$: $P(\tilde{x}|X)$. A posterior predictive distribution accounts for uncertainty about ${\displaystyle \theta }$. The posterior distribution of possible ${\displaystyle \theta }$ values depends on ${\displaystyle \mathbf {X} }$:

\begin{equation}
{\displaystyle p(\theta |\mathbf {X} )}
\end{equation}

The posterior predictive distribution of ${\displaystyle {\tilde {x}}}$ given ${\displaystyle \mathbf {X} }$  is calculated by marginalizing the distribution of ${\displaystyle {\tilde {x}}}$ given ${\displaystyle \theta }$  over the posterior distribution of ${\displaystyle \theta }$  given ${\displaystyle \mathbf {X} }$:

\begin{equation}
{\displaystyle p({\tilde {x}}|\mathbf {X} )=\int _{\Theta }p({\tilde {x}}|\theta ,\mathbf {X} )\,p(\theta |\mathbf {X} )\operatorname {d} \!\theta }
\end{equation}

Because it accounts for uncertainty about ${\displaystyle \theta }$, the posterior predictive distribution will in general be wider than a predictive distribution which plugs in a single best estimate for ${\displaystyle \theta }$ .

In practice, we approximate the predictive distribution $P(\tilde{x}|X)$ using sampling in two steps:

- __Step 1__: In the first step, we obtain a value of the model parameter $\theta_i$ by indepentently sampling from the posterior distribution: $\theta_i \sim P(\theta|X)$
- __Step 2__: Next, what we do is we use that value of $\theta_i$ and we plug that into the sampling distribution. Then, we sample a value of $\tilde{x}$, which is conditional on that value of $\theta_i$: $\tilde{x}_i \sim P(\tilde{X}|\theta_i)$.

The above process is then repeated for many times and if we draw a historgram of the $\tilde{x}_i$ values, we expect it to be an approximation to our posterior predicted distribution:


$\mathrm{for}\, i=1:\mathrm{many}\{$

$\qquad P(\tilde{X}|X) \quad \leftarrow \quad  \tilde{x}_i \sim P(\tilde{X}|\theta_i) \quad \leftarrow \quad \theta_i \sim P(\theta|X)$

$\}$


Note that, each of those two steps represent the two sources of uncertainty that we deal with in predicting new data. Firstly, there is uncertainty on the value of the parameter $\theta$ and so that is dictated by the posterior distribution. Secondly, there is the uncertainty in the data generating process itself; if we even didn't know what $\theta$ was, there is still a certain factor of randomness in that process. The latter source of uncertainty is represented by the second (middle) step of this sampling process.

## Example

To provide a thourough explanation of how this technique works, we are going to go through an example now. Let's start up a problem by imagining we have got a sample of $n$ school students, and in that sample we count the number that wear myopia glasses, $m$. We assume that always the number of students with near-sightedness are less or equal from the total population, i.e. $m \leq n$. 

To come up with a suitable likelihood we need to consider the following facts:

1. the data sample size is fixed
2. the observed outcome $X$ is a discrete random variable. In the case we are measuring the number of students whose vision condition exceed some given threshold, we say to have discrete random variable.

and the following assumptions:

1. the vision conditions of a student is independent of another student. This means that, if we know the vision condition of one student, it does not help us predict the vision condition of another student, apart from the fact that both come from the same population. This assumption will be violated if we take students from the same family, or they are twins, because you expect then those students to be more similar than children in the overall opulation. Supposing that, what we are interested here is inferring the myopia rate of school students in the population. 
2. the students come from the same poplulation. In other words, their data are identically and independently distributed (iid). This assumptio will be violated if some of the student coming from one population and another portion is coming from a different population.

By combining the above facts and assumptions together, we obtain a _binomial_ likelihood. So, in this example we are going to use a binomial probability mass function (pmf), assuming that $X$ is binomialy distributed with a sample size of $n$ and a parameter $p$:

\begin{equation}
\mathrm{B}(n,\,p)
\end{equation}

where $p=\theta$ represents the proportion of the population of students which wear myopia glasses. In general, if the random variable $X$ follows the binomial distribution with parameters $n \in \mathbb{N}$ and $p \in [0,\,1]$, we write $X \sim B(n,\, p)$. Then we can use a beta function 

\begin{equation}
{\displaystyle \beta (x,y)=\int _{0}^{1}t^{x-1}(1-t)^{y-1}\,dt}
\end{equation}

for complex number inputs $x$, $y$ such that Re $x > 0$, Re $y > 0$, as prior on $\theta$. The benefit of using a $\beta$ function prior is that is conjugated to a binomial likelihood, which means we can exactly calculate the posterior. We get that $\theta$ conditioned on $X$ 

\begin{equation}
\theta | X \sim \beta(a,\,b)
\end{equation}

is a $\beta$ distribution with the first argument being $a=1+m$ and the second argument being $b=1+n-m$. For example, if $n=10$ with $m=2$ then we obtain a beta $\beta(3,\,9)$ as our posterior. 

So, we are going to be sampling from a beta posterior distribution, 

\begin{equation}
\beta(a,\,b) \quad \mathrm{with} \quad a=1+m \quad \mathrm{and} \quad b=1+n-m
\end{equation}

to obtain a value of $\theta_i$. Hereafater, we use a binomial sampling distribution conditional on the obtained $\theta_i$ to sample a value of $\tilde{x}_i$. Also, we are going to assume in the example that the sample size in the predicted dataset is fixed to $n$.

## Simulation

Having the theory set, we can proceed to performing simulations to illustrate how we can build up the posterior predictive distribution by sampling. Let's first do the necessary imports of python libraries.

In [ ]:
import numpy as np
from scipy.stats import beta, binom
import matplotlib.pyplot as plt
from typing import List
from pathlib import Path
import imageio

Then we define a handy class to nicely bundle the posterior outcomes together.

In [ ]:
class Posterior:
    """
    Class to group the posterior results together
    for convenient handling in drawing the distributions
    afterwards
    """
    def __init__(self,
                 name:str=None,
                 a:float=None,
                 b:float=None,
                 x:float=None,
                 y:float=None,
                 rv:float=None):
        self.__name = name
        self.__a = a    # parameter a
        self.__b = b    # parameter b
        self.__x = x    # x data
        self.__y = y    # y data
        self.__rv = rv  # random value

    @property
    def name(self):
        return self.__name
        
    @property
    def a(self):
        return self.__a

    @property
    def b(self):
        return self.__b

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y
    
    @property
    def rv(self):
        return self.__rv

For each iteration event we store the sampling outcomes on the parameter $\theta$ and $\tilde{x}$ in the next class.

In [ ]:
class Sampling:
    """
    Keeps data from sampling process:
    - sample theta from posterior distribution
    - new X data conditioned on theta
    """
    def __init__(self,
                 θ:Posterior=None,
                 X:Posterior=None):
        self.__θ = θ
        self.__X = X

    @property
    def θ(self):
        return self.__θ

    @property
    def X(self):
        return self.__X

All results are then collected in the following vector class.

In [ ]:
class SamplingData:
    """
    Collection of Sampling data
    """
    def __init__(self,
                 i:int=None,
                 n:int=None,
                 m:int=None,
                 r:int=None,
                 ε:float=None):
        self.__i = i          #number of iterations
        self.__n = n          #size of population
        self.__m = m          #size of subset in population
        self.__r = r          #size of posterior sample to be drawn
        self.__ε = ε          #epsilon value to avoid irregularities at distribution edges
        self.__list = []      #list of results
        self.__X_tilde = None #sampled data conditional on theta 

    def __call__(self): #overloaded parentheses operator
        return self.__list
    
    def add(self, x:Sampling):
        self.__list.append(x)

    @property
    def i(self):
        return self.__i
        
    @property
    def n(self):
        return self.__n

    @property
    def m(self):
        return self.__m
    
    @property
    def r(self):
        return self.__r

    @property
    def ε(self):
        return self.__ε
    
    @property
    def X_tilde(self):
        if not self.__X_tilde:
            self.__X_tilde = [sample.X.rv for sample in self.__list]
        return self.__X_tilde

The next function is useful for drawing the single posterior distributions.

In [ ]:
# drawing helper
def draw_sample(idx=None,
                sample = None,
                X_tilde = None,
                i = None,
                ε = 1e-6,
                n = None,
                m = None,
                r = None,
                directory = None,
                figsize = (15,5),
                show_graphics = False,
                save_graphics = False,
                ):

    #for counting purposes
    idx += 1
    
    #prepare canvas
    fig, ax = plt.subplots(1, 3, figsize=figsize)
    #fig.tight_layout()

    #########################################
    #posterior function for unknown parameter
    #########################################

    #Display the probability density function (pdf) which is
    #the posterior from which we are sampling theta
    ax[0].plot(sample.θ.x,
               sample.θ.y,
               'r-',
               lw=2,
               alpha=0.6,
               label=f'${sample.θ.name}$ pdf')
    ax[0].set_xlabel('$\\theta$')
    ax[0].set_ylabel('probability density')
    ax[0].set_title(f'n={n}, X={m} $\Rightarrow '
                    f'p(\\theta|X)={sample.θ.name}({sample.θ.a},\\,{sample.θ.b})$')
    ax[0].set_xlim([0, 1])
    ax[0].set_ylim([0, None])
    ax[0].axvline(x=sample.θ.rv,
                  label=f'Random value = {sample.θ.rv:.2f}',
                  c='orange')
    ax[0].legend(loc='upper right', frameon=False)
    
    ########################################
    # data distribution conditional on theta
    ########################################
    ax[1].plot(sample.X.x,
               sample.X.y,
               'bo',
               lw=2,
               alpha=0.6,
           label='binomial pmf')
    ax[1].set_xlim([-0.5, r])
    ax[1].set_ylim([0, None])
    ax[1].set_xlabel('$\\tilde{X}$')
    ax[1].set_ylabel('probability')
    ax[1].set_title('$p(\\tilde{X}|\\theta_{%i})=%s(%i,\\,%.2f)$'%(idx,
                                                                sample.X.name,
                                                                sample.X.a,
                                                                sample.X.b))
    ax[1].vlines(sample.X.x,
                 0,
                 sample.X.y,
                 colors='b',
                 lw=5,
                 alpha=0.5)
    ax[1].axvline(x=sample.X.rv,
                  label=f'Random value = {sample.X.rv}',
                  c='orange')

    ax[1].legend(loc='upper right', frameon=False)

    #########################################################
    # sample data from data distribution conditional on theta
    #########################################################

    #plot randomly sampled value
    hy, hx, _ = ax[2].hist(X_tilde,
                           density=False,
                           bins=10,
                           alpha=0.6,
                           label='Samples')
    #define offset
    hy_max = hy.max()
    offset = 1.15
    
    ax[2].set_xlim([-0.5, r])
    ax[2].set_ylim([0, hy_max*offset])
    ax[2].set_xlabel('$\\tilde{X}$')
    ax[2].set_ylabel('count')
    ax[2].set_title('$p(\\tilde{X}|X)$ sample=%d/%d'%(idx, i))

    #indicate the bin in which the data was added
    ax[2].arrow(X_tilde[-1],
                hy_max*offset,
                0,
                -hy_max*0.05,
                length_includes_head=False,
                head_width=0.5,
                head_length=hy_max*0.05)
    
    #iteration number
    number_str = str(idx)
    zero_filled_number_str = number_str.zfill(len(str(i)))

    #filename
    path = Path(directory)
    
    figname = path.joinpath('sampling_%s.png'%(zero_filled_number_str) )

    #show graphics
    if show_graphics: plt.show()

    #store graphics
    if save_graphics: fig.savefig(figname, dpi=fig.dpi)

    #release stuff from memory
    plt.close('all')

The following function calls the previous one to create a bunch of posterior predictive distribtions obtained from sampling.

In [ ]:
def draw(samples:SamplingData=None,
         directory:str=None):

    #folder to keep
    path = Path(directory)
    path.mkdir(parents=True,
               exist_ok=True)

    for f in path.glob('*.*'):
        f.unlink()
    
    #collect drawn posterior data in an increamental way
    #so to monitor progress
    X_tilde_increamental = []
    
    for isample, sample in enumerate(samples()):
        X_tilde_increamental.append(sample.X.rv)
        draw_sample(idx = isample,
                    sample = sample,
                    X_tilde = X_tilde_increamental,
                    i = samples.i,
                    n = samples.n,
                    m = samples.m,
                    ε = samples.ε,
                    r = samples.r,
                    directory = directory,
                    show_graphics=False,
                    save_graphics=True
                    )

The following function helps to use each plot as frame and create an animation.

In [ ]:
def animate(directory:str):

    image_path = Path(directory)
    images = list(image_path.glob('*.png'))
    frames = []
    for file_name in sorted(images):
        frames.append(imageio.imread(file_name))

    exportfile = image_path.joinpath('animation.gif')
    #imageio.mimwrite(exportfile,
    #                 frames,
    #                 fps= 100 # number of frames per second (default=10) 
    #                 )

    imageio.mimsave(exportfile,
                    frames,
                    format='GIF',
                    #duration=0.75,
                    fps=1 # frames per second: the smaller, the faster
                    )
    #clean up local files
    for image in images:
        image.unlink()

    print('Results in file:', exportfile)

Finally, we define the iterative sampling process method and the  main user-callable function for running that method. 

In [ ]:
def sampling_process(sampling_data:SamplingData=None, #our sampling data
                     Niter:int=None, #number of events
                     n:int=None,     #population size
                     m:int=None,     #subset in population
                     r:int=None,     #size of distr to be drawn
                     ε=1e-5 #distribution edge offset
                     ):
    # fix the random seed for replicability.
    np.random.seed(33)
    
    #fixed beta parameters
    a_beta, b_beta = 1+m, 1+n-m

    #fixed X data for beta
    #use percent point function/ppf (inverse of cdf) at q for given params
    x_beta = np.linspace(beta.ppf(0+ε, a_beta, b_beta),
                         beta.ppf(1-ε, a_beta, b_beta),
                         100)

    #fixed y data for beta
    y_beta = beta.pdf(x_beta, a_beta, b_beta)

    #iterations
    for i in range(Niter):

        #generate random number from beta posterior distribution
        rv_beta = beta.rvs(a_beta, b_beta, size=1)[0]

        #our posterior for theta
        posterior_θ = Posterior(name='\\beta',
                                a=a_beta,
                                b=b_beta,
                                x=x_beta,
                                y=y_beta,
                                rv=rv_beta
                                )
    
        #binomial distribution conditional on theta - varying params
        a_binom, b_binom = r, rv_beta

        #binomial X data - varying data
        x_binom = np.arange(binom.ppf(0+ε, a_binom, b_binom),
                            binom.ppf(1-ε, a_binom, b_binom))

        y_binom = binom.pmf(x_binom, a_binom, b_binom)
    
        #generate random number from binomial distribution: x_tilde
        rv_binom = binom.rvs(a_binom, b_binom, size=1)[0]
    
        #our new data conditioned on sampled theta
        posterior_X = Posterior(name='\\mathrm{Binomial}',
                                a=a_binom,
                                b=b_binom,
                                x=x_binom,
                                y=y_binom,
                                rv=rv_binom
                                )

        #keep our finding in a list
        sampling_data.add( Sampling(θ = posterior_θ,
                                X = posterior_X) )

In [ ]:
def run(Niter:int=None,     #number of events
        directory:str=None, #output dir
        n:int=None,         #population size
        m:int=None,         # subset in population
        r:int=None,         # size of population to be drawn
        ε=1e-5              #distribution edge offset
       ):
    
    # rename directory to avoid name clashing
    directory += f'_i{Niter}_n{n}_m{m}_r{r}'
    
    #list of sampling results
    sampling_data = SamplingData(i=Niter, n=n, m=m, r=r, ε=ε)
    
    #sampling
    sampling_process(sampling_data=sampling_data,
                     Niter=Niter,
                     n=n,
                     m=m,
                     r=r)
    
    # draw sampling data
    draw(samples=sampling_data,
         directory=directory)

    # animate
    animate(directory=directory)

Now we have all tools in our hands to run our first example on drawing a posterior predictive distribution by sampling. In this example, we set $n=10$ and $m=2$. We repeat our sampling trials by 50 times.

In [ ]:
#run the experiment
run(Niter = 50,
    directory = 'results/posterior_predictive_distr_by_sampling',
    n = 10,
    m = 2,
    r = 10)

![Posterior predictive distribution by sampling](results/posterior_predictive_distr_by_sampling_i50_n10_m2_r10/animation.gif)

On the left-hand plot, we have the posterior distribution, which is a $\beta(3,\,9)$ distribution. In the middle plot, we show a binomial probability distribution which is conditional on the $\theta$ value we sample from the $\beta$ posterior distribution. The value sampled from the $\beta$ posterior in each trial is indicated by a vertical orange line. After sampling the $\theta$ value from the posterior, we can plot what a binomial distribution with a sample size of $n=10$ and a probability of $\theta$ looks like. What we do next is to sample a value of $\tilde{X}$ from the corresponding binomial distribution. This is indicated again by a vertical orange line passing through the bin from which the value of $\tilde{X}$ is extracted and which goes into building up the histogram of our posterior predictive distribution on the right. 

For every iteration, we build up a new binomial distribution depending on the value of $\theta$ which is randomly obtained from the posterior $\beta$ distribution. Then in turn we sample a new value $\tilde{X}$ conditional on the sampling distribution assuming that $\theta$. As we do that, we build up an approximated posterior predictive distribution which starts to converge to a given distributional shape, as shown on the right.

Because the posterior $\beta$ distribution is centered on about 0.2, our posterior predictive distribution is unsurprisingly centered on about $m/n=2/10=0.2$. So, we would predict about 2 individuals carrying glasses out of 10 if we are to collect a sample of data.

Let's now illustrate this process for a different posterior  distribution. We can assume we have collected a sample size of $n=1000$ students and out of those we found $m=200$ of them wearing myopia glasses. If we use the same uniform prior as we used before, then we obtain a $\beta$ posterior distribution with $a=1+m=201$ and $b=1+n-m=801$, i.e. $\beta(201,\, 801)$. Because we have collected a much bigger sample size, our posterior is going to be much more narrow in its core region around $\theta=0.2$.

In [ ]:
#run the experiment with a different posterior
run(Niter = 50,
    directory = 'results/posterior_predictive_distr_by_sampling',
    n = 1000,
    m = 200,
    r = 10)

![Posterior predictive distribution by sampling](results/posterior_predictive_distr_by_sampling_i50_n1000_m200_r10/animation.gif)

When we run this process, we observe that the values of $\theta$ we are sampling tend to be very close to one another and centered around 0.2. Consequently, our binomial sampling distribution isn not changing drastically between iterations. Even though we have collected an bigger original sample of 1000 students, we can still predict the number of students out of 10 in our new, predicted sample in order to be able to read off and compare. On the right, we are again showing our predictive posterior distribution which is assuming that in the new sample has size $n=10$.

We observe that after some time, the posterior predictive distribution takes a certain distributional shape and it converges to a distribution which looks similar to our sampling distribution.

To compare, we can show the two examples one on top of the other as presented below.

![Posterior predictive distribution by sampling](results/posterior_predictive_distr_by_sampling_i50_n10_m2_r10/animation.gif)
![Posterior predictive distribution by sampling](results/posterior_predictive_distr_by_sampling_i50_n1000_m200_r10/animation.gif)

The top row of plots corresponds to the first example where we obtained a sample size of $(n=10,\,m=2)$ whereas the bottom plots represent the outcome for a sample size of $(n=1000,\,m=200)$. 

We can see that the posterior between these two examples is quite different with the second one being obviously much more peaky. However, on the right histograms we do not notice such big differences in the posterior predictive distribution. Of course, the bottom right data histogram is slightly more narrow and peaked at 2, but there aren't that marked differences between the two posterior predictive distributions. What could be the underlying reason for that? The posterior predictive distribution represent the uncertainty over two different causes:

1. the uncertainty over the value of the unknown parameter $\theta$. This is represented by the posterior in the first step when we sample a value $\theta$ from the posterior.
2. the uncertainty coming from the binomial distribution itself. The binomial distribution we sample from is relatively insensitive to the value of $\theta$ that we are sampling across these two examples.

The posterior predictive distribution is overall representing mostly the uncertainty in the sampling distribution shown in the middle by the binomial, because that tends to be the dominant source of uncertainty. 

## Summary

The posterior predictive distribution is a distribution that tends to be too difficult to calculate analytically. Therefore, in most applied circumstances we obtain an approximate posterior predictive distribution using sampling. In the first step, we sample a value of $\theta$ from a posterior distribution and then conditional on that value of theta we sample a value of data in a new sample, $\tilde{X}$. If this process is repeated a large number of times, we then obtain an approximate posterior predictive distribution. The posterior predictive distribution is representative of the uncertainty of two sources: (a) the source of uncertainty over the value of the parameter $\theta$ and (b) the source of uncertainty coming from the sampling distribution itself, which represents the uncertainty over the process which generates data. 

# References

- Posterior predictive distribution, [Wikipedia article](https://en.wikipedia.org/wiki/Posterior_predictive_distribution)

- A Student's Guide to Bayesian Statistics, Ben Lambert, [Book on Amazon](https://www.amazon.de/-/en/Ben-Lambert/dp/1473916364/ref=sr_1_1?dchild=1&keywords=A+Student%27s+Guide+to+Bayesian+Statistics&qid=1609326094&sr=8-1), Chapter 7.8. Youtube [channel](https://www.youtube.com/playlist?list=PLwJRxp3blEvZ8AKMXOy0fc0cqT61GsKCG).